In [1]:
import matplotlib.pyplot as plt
import numpy as np
from collections import defaultdict
from main import parse_config, instantiate_agents, instantiate_auction
from tqdm.notebook import tqdm

In [2]:
# Parse configuration file
rng, config, agent_configs, agents2items, agents2item_values,\
num_runs, max_slots, embedding_size, embedding_var,\
obs_embedding_size = parse_config('../config/FP_DM_Oracle.json')

In [7]:
# Placeholders for output
auction_revenue = []
social_welfare = []
social_surplus = []
# Instantiate Agent and Auction objects
# Context comes from the embedding var and size to generate a normal distribution
agents = instantiate_agents(rng, agent_configs, agents2item_values, agents2items)

In [42]:
auction, num_iter, rounds_per_iter, output_dir =\
    instantiate_auction(rng,
                        config,
                        agents2items,
                        agents2item_values,
                        agents,
                        max_slots,
                        embedding_size,
                        embedding_var,
                        obs_embedding_size)

In [43]:
self = auction

In [44]:
import numpy as np

from BidderAllocation import OracleAllocator
from Models import sigmoid

In [45]:
num_slots = self.rng.integers(1, self.max_slots + 1)

# Sample a true context vector
# The true context vector is the same for all agents
# the last element is always 1.0 to account for the bias
true_context = np.concatenate((self.rng.normal(0, self.embedding_var, size=self.embedding_size), [1.0]))

# Mask true context into observable context
obs_context = np.concatenate((true_context[:self.obs_embedding_size], [1.0]))

# At this point, the auctioneer solicits bids from
# the list of bidders that might want to compete.
bids = []
CTRs = []
participating_agents_idx = self.rng.choice(len(self.agents), self.num_participants_per_round, replace=False)
participating_agents = [self.agents[idx] for idx in participating_agents_idx]
for agent in participating_agents:
    # Get the bid and the allocated item
    # OracleAllocator needs to know the true context
    if isinstance(agent.allocator, OracleAllocator):
        bid, item = agent.bid(true_context)
    else:
        bid, item = agent.bid(obs_context)
    bids.append(bid)
    # Compute the true CTRs for items in this agent's catalogue
    true_CTR = sigmoid(true_context @ self.agent2items[agent.name].T)
    agent.logs[-1].set_true_CTR(np.max(true_CTR * self.agents2item_values[agent.name]), true_CTR[item])
    CTRs.append(true_CTR[item])
bids = np.array(bids)
CTRs = np.array(CTRs)

In [46]:
self

In [49]:
self.agents[1].select_item(true_context)

(7, 0.01956650560803386)

In [32]:
self = agent.bidder

In [33]:
n_values_search = 128
gamma_grid = self.rng.uniform(0.1, 1.0, size=n_values_search)

In [36]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats
import torch

from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF
from sklearn.metrics import roc_auc_score
from tqdm import tqdm

from Impression import ImpressionOpportunity
from Models import BidShadingContextualBandit, BidShadingPolicy, PyTorchWinRateEstimator

In [38]:
gamma_grid.sort()
x = torch.Tensor(np.hstack((np.tile(0.5, (n_values_search, 1)), np.tile(0.6, (n_values_search, 1)), gamma_grid.reshape(-1,1))))

In [40]:

n_values_search

128